In [ ]:
# This cell does not get executed when run with Times Square
import os
import datetime

visit_origin = os.environ.get("SCHEDVIEW_VISIT_ORIGIN", "lsstcam")
day_obs = int(
    os.environ.get(
        "SCHEDVIEW_DAY_OBS",
        (datetime.date.today() - datetime.timedelta(days=1)).isoformat(),
    ).replace("-", "")
)
extrap_day_obs = int(
    os.environ.get(
        "EXTRAP_DAY_OBS",
        "20261101"
    )
)

In [ ]:
"""
# To render this notebook to html, in the current directory with the current python environment

export ACCESS_TOKEN_FILE=${HOME}/.lsst/usdf_access_token
export SCHEDVIEW_VISIT_ORIGIN=lsstcam
export SCHEDVIEW_DAY_OBS='20260204'
export EXTRAP_DAY_OBS='20261101'
time jupyter nbconvert \
    --to html \
    --execute \
    --no-input \
    --ExecutePreprocessor.kernel_name=python3 \
    --ExecutePreprocessor.startup_timeout=3600 \
    --ExecutePreprocessor.timeout=3600 \
    pre-progress.ipynb

# The public USDF page served at
#    https://s3df.slac.stanford.edu/data/rubin/sim-data/schedview/reports/preprogress/lsstcam/2026/02/04/pre-progress_2026-02-04.html
# is found at
#    /sdf/group/rubin/web_data/sim-data/schedview/reports/preprogress/lsstcam/2026/02/04/pre-progress_2026-02-04.html

# You can set up the conda environment used to automatically generate these pages with
source /sdf/group/rubin/sw/w_latest/loadLSST.sh
conda activate /sdf/data/rubin/shared/scheduler/envs/prenight
"""
pass

In [ ]:
import os
import sys

#sched_source = 'env'
#sched_source = 'shared'
sched_source = 'devel'
match sched_source:
    case 'shared':
        if os.path.exists('/sdf/data/rubin/shared/scheduler/packages'):
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/uranography-1.4.1")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/rubin_scheduler-3.22.0")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/rubin_sim-2.6.1")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/schedview-0.22.0a2")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/rubin_nights-0.11.0")
    case 'devel':
        if os.path.exists('/sdf/data/rubin/user/neilsen/devel'):
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_scheduler")
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_sim")
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/schedview")
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/uranography")
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_nights")
    case _:
        # Use whatever is in the kernel python environment
        pass

In [ ]:
from schedview.util import config_logging_for_reports
import logging
config_logging_for_reports(logging.ERROR)

In [ ]:
import calendar
import datetime
import os
import sqlite3
import sys
from collections import defaultdict, OrderedDict
from functools import partial
from pathlib import Path
from collections import ChainMap
from tempfile import TemporaryDirectory
from collections.abc import Mapping

import astropy
import astropy.units as u
import bokeh
import bokeh.io
import bokeh.layouts
import bokeh.models
import bokeh.plotting
import bokeh.transform
import cartopy
import colorcet
import healpy
import healpy as hp
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from erfa import ErfaWarning
from astropy.coordinates import SkyCoord, get_body
from astropy.time import Time
from astropy.visualization import ZScaleInterval
from IPython.display import HTML, display, Markdown, HTML
from lsst.resources import ResourcePath

In [ ]:
usdf_sim_data_dir = "/sdf/data/rubin/shared/rubin_sim_data"
if os.path.exists(usdf_sim_data_dir):
    os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"

In [ ]:
import schedview.collect.consdb
import schedview.collect.nightreport
import schedview.collect.timeline
import rubin_scheduler
import rubin_scheduler.site_models
import rubin_scheduler.utils
import schedview.compute.maf
import schedview.compute.astro
import schedview.compute.visits
import schedview.compute.nightreport
import schedview.plot.survey_skyproj
import schedview.plot.visitmap
import schedview.plot.nightreport
import schedview.plot.progress
import schedview.plot.monthlyhg
import schedview.collect.visits
import schedview.collect.sv
import rubin_sim.sim_archive
import uranography
from rubin_sim.data import get_baseline
from rubin_scheduler.scheduler.model_observatory import ModelObservatory
from rubin_sim import maf
from schedview.compute.camera import LsstCameraFootprintPerimeter
from uranography.api import Planisphere, make_zscale_linear_cmap
from schedview import DayObs
from rubin_scheduler.utils.consdb import KNOWN_INSTRUMENTS
from rubin_nights.reference_values import SCIENCE_PROGRAMS
from schedview.plot.survey_skyproj import map_hpix_in_laea_and_mcbryde, map_metric_in_laea_and_mcbryde
from schedview.plot import PLOT_BAND_COLORS, LIGHT_PLOT_BAND_COLORS, LIGHT_EXTRA_COLORS
from schedview.plot import mpl_fig_to_html


In [ ]:
if not Path('/sdf/group/rubin/web_data/sim-data').exists():
    schedview.collect.visits.OPSIMDB_TEMPLATE = str(
        Path.home().joinpath("Data/sim-data/sims_featureScheduler_runs{sim_version}/baseline/baseline_v{sim_version}_10yrs.db")
    )

In [ ]:
# Validate the input
import re

assert visit_origin in KNOWN_INSTRUMENTS or visit_origin == 'baseline' or re.match(r"^\d+\.\d+$", visit_origin) is not None

In [ ]:
# Degraded IERS accuracy is never going to be important for these figures.

# If IERS degraded accuracy encountered, don't fail, just keep going.
astropy.utils.iers.conf.iers_degraded_accuracy = "ignore"

In [ ]:
bokeh.io.output_notebook(hide_banner=True)

In [ ]:
%matplotlib inline

In [ ]:
ORIGIN_TELESCOPE = defaultdict(
    np.array(['Simonyi']).item,
    {'latiss': 'AuxTel'}
)

In [ ]:
observatory = ModelObservatory(no_sky=True)
timezone = "Chile/Continental"
telescope = ORIGIN_TELESCOPE[visit_origin]
from_opsim = visit_origin not in KNOWN_INSTRUMENTS
science_programs_to_include = SCIENCE_PROGRAMS
telescope = "AuxTel" if visit_origin.lower()=="latiss" else "Simonyi"

In [ ]:
day_obs = DayObs.from_date(day_obs)
observatory.mjd = day_obs.mjd + 1 - observatory.location.lon.deg/360 ;# The approximate middle of the night

In [ ]:
extrap_day_obs = DayObs.from_date(extrap_day_obs)

In [ ]:
conditions = observatory.return_conditions()

In [ ]:
if True:
    visit_cache = '/sdf/data/rubin/user/neilsen/devel/ehn_notebooks_2026/test_visits.h5'
    if Path(visit_cache).exists():
        visits = pd.read_hdf(visit_cache, 'visits')
        survey_visits = pd.read_hdf(visit_cache, 'survey_visits')
        baseline_visits = pd.read_hdf(visit_cache, 'baseline_visits')
    else:
        visits = schedview.collect.visits.read_visits(day_obs, 'lsstcam', stackers = schedview.collect.visits.NIGHT_STACKERS, num_nights=4000)
        visits.to_hdf(visit_cache, 'visits')
        survey_visits = visits.loc[visits['science_program'].isin(SCIENCE_PROGRAMS) & visits['fiveSigmaDepth'].notna(), :]  if visit_origin == 'lsstcam' else visits
        survey_visits.to_hdf(visit_cache, 'survey_visits')
        baseline_visits = schedview.collect.visits.read_visits(extrap_day_obs, 'baseline',  stackers = schedview.collect.visits.NIGHT_STACKERS, num_nights=4000)
        baseline_visits.to_hdf(visit_cache, 'baseline_visits')
else:
    visits = schedview.collect.visits.read_visits(day_obs, visit_origin, stackers = schedview.collect.visits.NIGHT_STACKERS, num_nights=4000)
    survey_visits = visits.loc[visits['science_program'].isin(science_programs_to_include), :] if visit_origin == 'lsstcam' else visits
    baseline_visits = schedview.collect.visits.read_visits(extrap_day_obs, 'baseline',  stackers = schedview.collect.visits.NIGHT_STACKERS, num_nights=4000)

In [ ]:
night_events = schedview.compute.astro.night_events(day_obs.date)

# Pre-start Survey Progress

This report provides examples of the kinds of figures that can be created in a survey progress report for Rubin Observatory's LSST.

In most cases, the report generation tools use [maf](https://www.lsst.org/scientists/simulations/maf) to compute metrics and either `schedview` or `maf` (or a combination of the two) to make the figures.

The specific metrics used are merely examples of the kinds of metrics that can be shown in each type of figure: we expect to include additional metrics based on community and project feedback.

Most `maf` metrics are candidates for inclusion, and additional metrics can be contributed.

***The data shown in these plots is not a true representation of progress on the LSST survey***: all visits taken thusfar are included in the "completed" visits shown in these plots, and these are compared to a baseline in which the survey starts in November of 2026.
Neither of these data sets represent actual progress on the actual LSST survey: the actual "start date" of the LSST survey is to be determined, as is the selection of visits taken thusfar to be included in progress (if any).

## Progress as measured by scalar ("summary") metrics

**Any MAF summary metric (such as those reported [here](https://usdf-maf.slac.stanford.edu/summaryStats?runId=1)) can potentially be computed and reported on either of these plots, up to any cutoff date,** such as the cutoff for inclusion in DR1, or the end of the 10 year survey.

Two examples are shown below:
 - the total number of science visits i, which shows the advantages plotting the instantaneous metric value vs. date.
 - the total area with more than 50 visits in square degrees (f0Area), an example for which the extrapolated final value is a more useful measure of progress toward the final objective.

In the examples here, metrics are extrapolated to 2026-11-01.

In [ ]:
BENCHMARK_AREA = 18000
MIN_N_VISITS = 50
NSIDE = 16
BAND = 'i'
BANDS = 'ugrizy'
OBS_MJD_COLUMN = 'observationStartMJD'

In [ ]:
start_dayobs = DayObs.from_time(np.floor(baseline_visits['observationStartMJD'].min()))

In [ ]:
metric_name = f"fOArea{MIN_N_VISITS}"

# Make functions to make new slicers, metrics, and summary metrics,
# because I'm not sure the same instances can be reused.
make_slicer = partial(maf.HealpixSlicer, nside=NSIDE, verbose=False)
make_metric = partial(maf.CountExplimMetric, metric_name="fO")
make_summary_metric = partial(
    maf.FOArea,
    nside=NSIDE,
    norm=False,
    metric_name=metric_name,
    asky=BENCHMARK_AREA,
    n_visit=MIN_N_VISITS
)

metric_values = schedview.compute.maf.make_metric_progress_df(
    survey_visits,
    baseline_visits,
    start_dayobs=start_dayobs,
    extrapolation_dayobs=extrap_day_obs,
    slicer_factory=make_slicer,
    metric_factory=make_metric,
    summary_metric_factory=make_summary_metric,
    freq={'completed': 'W', 'future': 'MS'},
)

figs = schedview.plot.progress.make_metric_line_plots(metric_name, metric_values)
bokeh.io.show(figs)

In [ ]:
metric_name = f"N_visits_{BAND}"

# Make functions to make new slicers, metrics, and summary metrics,
# because I'm not sure the same instances can be reused.
make_metric = partial(maf.CountMetric, OBS_MJD_COLUMN, metric_name=metric_name)
query = f"band == '{BAND}'"

metric_values = schedview.compute.maf.make_metric_progress_df(
    survey_visits.query(query),
    baseline_visits.query(query),
    start_dayobs=start_dayobs,
    extrapolation_dayobs=extrap_day_obs,
    slicer_factory=maf.UniSlicer,
    metric_factory=make_metric,
    freq={'completed': 'W', 'future': 'MS'},
)

figs = schedview.plot.progress.make_metric_line_plots(metric_name, metric_values)
bokeh.io.show(figs)

## Metric maps

Each subplot represents the map of a metric a different filter, presented in a [Lambert Azimuthal Equal Area Projection](https://en.wikipedia.org/wiki/Lambert_azimuthal_equal-area_projection), centered at the south celestial pole.

**Any MAF metric that uses a healpix slicer can be added to this set of maps.**

Annotations are similar to those of the "Visit Map" above:

 - The cividis (yellow to dark blue) backdrop represents the value of the metric.
 - The orange disk shows the coordinates of the moon.
 - The yellow disk shows the coordinates of the sun.
 - The green line (oval) shows the ecliptic.
 - The blue line (oval) shows the plane of the Milky Way.

### Accumulated numbers of visits

In [ ]:
survey_visits_with_depth = survey_visits.loc[~ np.isnan(survey_visits.eff_time_median.values.astype(float)), :]

In [ ]:
fig_visit_positions = {"Total number of visits accumumlated so far": np.where(survey_visits_with_depth.observationStartMJD > 0)[0]}
fig_visit_positions.update({f"Total accumulated visits in {b} band": np.where(survey_visits_with_depth.band == b)[0] for b in BANDS})

with plt.ioff():
    for summary, positions in fig_visit_positions.items():
        if len(positions) > 0:
            fig = map_metric_in_laea_and_mcbryde(
                survey_visits_with_depth.iloc[positions, :],
                maf.CountMetric(col='band'),
                schedview.plot.survey_skyproj.map_healpix,
                observatory,
                night_events,
                lon_0=night_events.loc['night_middle', 'LST'].item(),
                figsize=(15, 5),
                cmap='cividis_r',
                horizons=None,
            )
            map_html = mpl_fig_to_html(fig, summary=summary)
            display(HTML(map_html))

### Map of depth (inverse variance of noise) for ideal coadd of visits accumulated so far

The following map shows accumulated inverse variance of completed visits, a uniformly increasing measure of progress toward a target limiting coadd limiting magnitude. The inverse variance of the noise is increases linearly with the number of exposures for exposures of uniform depth, and total exposure time when noise is dominated by sky brightness, and so is sometimes scaled to a time under reference conditions and called the "effective exposure time." See DMTN-296: Calculations of Image and Catalog Depth for more details.


In [ ]:
fig_visit_positions = {f"Accumulated inverse noise variance (t<sub>eff</sub>) in {b} band": np.where(survey_visits_with_depth.band == b)[0] for b in BANDS}

with plt.ioff():
    for summary, positions in fig_visit_positions.items():
        if len(positions) > 0:
            fig = map_metric_in_laea_and_mcbryde(
                survey_visits_with_depth.iloc[positions, :],
                maf.SumMetric(col='eff_time_median'),
                schedview.plot.survey_skyproj.map_healpix,
                observatory,
                night_events,
                lon_0=night_events.loc['night_middle', 'LST'].item(),
                figsize=(15, 5),
                cmap='cividis_r',
                horizons=None)
            map_html = mpl_fig_to_html(fig, summary=summary)
            display(HTML(map_html))         

## DDF Cadence

In [ ]:
time_window_duration=90

In [ ]:
try:
    ddf_visits = schedview.collect.visits.read_ddf_visits(day_obs, visit_origin, num_nights=time_window_duration)
    ddf_visits = ddf_visits.loc[ddf_visits['science_program'].isin(science_programs_to_include), :]
except:
    ddf_visits = []

if len(ddf_visits) > 0:
    nightly_ddf = schedview.compute.visits.accum_stats_by_target_band_night(ddf_visits, target_column='field_name')
    cadence_plots = schedview.plot.create_cadence_plot(
        nightly_ddf, day_obs.mjd - time_window_duration, day_obs.mjd, target_column='field_name'
    )
    bokeh.io.show(cadence_plots)
else:
    print("No DDF visits")


The y-axis (height of the vertical bars) represents the accumulated effective exposure time, $t_{\mbox{eff}}$ (as defined [DMTN-296](https://dmtn-296.lsst.io/)) accumulated over all exposures on the field for the night, colored by filter.

## Hourglass plots

In the hourglass plots that follow, each horizontal band represents a timeline for a day of a month. The gray background shows twilight, while the black background is fully night.
The moon rises or sets where the dotted line crosses each night's horizontal line, and transits where the thick solid line crosses it.

The quantities shown here are just a few examples. Similar hourglass plots can be added for any quantity that varies by visit, and whose value can be mapped to a scalar or category.


In [ ]:
first_date, last_date = [DayObs.from_time(m).date for m in visits.observationStartMJD.describe()[['min', 'max']]]

### Visit purpose (LSST survey or other?)

In [ ]:
name = "visit purpose"

def default_program():
    return "Other"

program_mapping = defaultdict(default_program)
for program in SCIENCE_PROGRAMS:
    program_mapping[program] = "LSST Survey"

metric_bundle = maf.metric_bundles.MetricBundle(
    metric=maf.ApplyMappingMetric('science_program', mapping=program_mapping),
    slicer=maf.slicers.VisitIntervalSlicer(),
    plot_dict={
        "cmap": plt.get_cmap("Set1"),
        "legend_ncols": 1,
        "legend_loc": (1.01, 0.5),
        "legend_bbox_to_anchor": (1.01, 0.0),
        "legend": True,
    }
)
with TemporaryDirectory() as maf_dir:
    maf.MetricBundleGroup([metric_bundle], None, out_dir=maf_dir).run_current(None, visits.to_records(index=False))


these_plots_html = schedview.plot.monthlyhg.plot_collapsed_monthly_hourglass_metric(metric_bundle, name, first_date, last_date, maf.plots.MonthHourglassCategoricalPlot)
display(HTML(these_plots_html))

### Seeing

In [ ]:
name = "seeingFwhmEff"
metric_bundle = maf.metric_bundles.MetricBundle(
    metric=maf.metrics.MedianMetric("seeingFwhmEff"),
    slicer=maf.slicers.VisitIntervalSlicer(),
)
with TemporaryDirectory() as maf_dir:
    maf.MetricBundleGroup([metric_bundle], None, out_dir=maf_dir).run_current(None, visits.to_records(index=False))

these_plots_html = schedview.plot.monthlyhg.plot_collapsed_monthly_hourglass_metric(metric_bundle, name, first_date, last_date, maf.plots.MonthHourglassPlot)
display(HTML(these_plots_html))

### Hour Angle

In [ ]:
name = "Hour Angle"
metric_bundle = maf.metric_bundles.MetricBundle(
    metric=maf.metrics.MedianMetric("HA"),
    slicer=maf.slicers.VisitIntervalSlicer(),
    plot_dict={"cmap": plt.get_cmap("coolwarm"), "color_limits": (-4.5, 4.5)}
)
with TemporaryDirectory() as maf_dir:
    maf.MetricBundleGroup([metric_bundle], None, out_dir=maf_dir).run_current(None, visits.to_records(index=False))

these_plots_html = schedview.plot.monthlyhg.plot_collapsed_monthly_hourglass_metric(metric_bundle, name, first_date, last_date, maf.plots.MonthHourglassPlot)
display(HTML(these_plots_html))

### Band

In [ ]:
name = "Band"
all_bands = list(LIGHT_PLOT_BAND_COLORS) + [b for b in visits.band.unique() if b not in LIGHT_PLOT_BAND_COLORS]
metric_bundle = maf.metric_bundles.MetricBundle(
    metric=maf.ApplyMappingMetric('band'),
    slicer=maf.slicers.VisitIntervalSlicer(),
    plot_dict={
        "cmap": mpl.colors.ListedColormap(LIGHT_EXTRA_COLORS, 'band'),
        "assigned_colors": OrderedDict((c, i) for i, c in enumerate(all_bands)),
        "legend_ncols": 1,
        "legend_loc": (1.01, 0.5),
        "legend_bbox_to_anchor": (1.01, 0.0),
        "legend": True,
    }
)
with TemporaryDirectory() as maf_dir:
    maf.MetricBundleGroup([metric_bundle], None, out_dir=maf_dir).run_current(None, visits.to_records(index=False))

these_plots_html = schedview.plot.monthlyhg.plot_collapsed_monthly_hourglass_metric(metric_bundle, name, first_date, last_date, maf.plots.MonthHourglassCategoricalPlot)
display(HTML(these_plots_html))
